Parse all the data

In [1]:
import os
import pandas as pd
from pprint import pprint
import numpy as np

In [2]:
# load in all the data in Datasets directory
folder_name = "Datasets"
csv_files = [f for f in os.listdir(folder_name) if f.endswith(".csv")]
dfs = {}
for file in csv_files:
    dfs[file.replace(".csv", "")] = pd.read_csv(os.path.join(folder_name, file))

In [3]:
# get the dataset by index
def get_i_dataset(i):
    return dfs[list(dfs.keys())[i]]


# print first 5 rows of the dataset
def print_dataset(i):
    print(list(dfs.keys())[i])
    print(dfs[list(dfs.keys())[i]].head())

'''
convert latitude and longitilde data to np array of location pairs  
'''
def convert_to_numpy(df):
    df[["Latitude", "Longitude"]] = df[["Latitude", "Longitude"]].apply(pd.to_numeric, errors='coerce')
    df = df.dropna().reset_index(drop=True)
    np_array = df[["Latitude", "Longitude"]].to_numpy()
    assert np_array.shape[1] == 2
    assert np_array.dtype == np.float64
    return np_array, df

In [4]:
NUM_OF_NEIGHBORHOODS = 90


def drop_na_check(df):
    df = df.dropna().reset_index(drop=True)
    assert len(df) == NUM_OF_NEIGHBORHOODS
    return df


def print_na(df):
    # print row containing NaN
    print(df[df.isna().any(axis=1)])

In [5]:
# construct new dataset with all data
df_output = dfs["neighborhood_area"][["Neighborhood", "Latitude", "Longitude"]]
df_output = drop_na_check(df_output)
df_output.sort_values(by=["Neighborhood"], inplace=True)
df_output = df_output.reset_index(drop=True)
neighborhood_loc, _ = convert_to_numpy(df_output)

In [6]:
def nearst_neighborhood(node):
    dist_2 = np.sum((neighborhood_loc - node) ** 2, axis=1)
    return np.argmin(dist_2)


def append_to_df_output(df, column_name, new_column_name):
    location, df = convert_to_numpy(df)
    neighborhoods = np.zeros(NUM_OF_NEIGHBORHOODS)
    for i, loc in enumerate(location):
        if not column_name:
            neighborhoods[nearst_neighborhood(loc)] += 1
        else:
            neighborhoods[nearst_neighborhood(loc)] += df.at[i, column_name]

    df_output[new_column_name] = neighborhoods

In [7]:
# adding arrests data
df = dfs['arrests']['Neighborhood'].sort_values()
# squash all row with the same name and make a new column "count"
df = df.groupby(df).size().reset_index(name='arrests_count')
df_temp = pd.merge(df_output, df, on='Neighborhood', how='left')
mt_oliver_count = df[df['Neighborhood'] == 'Mt. Oliver Neighborhood']['arrests_count'].sum()
mt_oliver_count += df[df['Neighborhood'] == 'Mount Oliver']['arrests_count'].sum()
df_temp.at[df_temp.index[df_temp['Neighborhood'] == 'Mt. Oliver'][0], 'arrests_count'] = mt_oliver_count
df_output = drop_na_check(df_temp)

In [8]:
# bike_stations
df = dfs['bike_stations']

In [9]:
# bike_stations
append_to_df_output(df, 'Total Docks', 'bike_station_count')

In [10]:
# crash_data
df = dfs['crash_data'].rename(columns={'DEC_LAT': 'Latitude', 'DEC_LONG': 'Longitude'})
append_to_df_output(df, '', 'crash_count')

In [11]:
# firearm seizure
df = dfs['firearm_seizure_data']['Neighborhood']
df = df.groupby(df).size().reset_index(name='firearm_seizure_count')

df_temp = pd.merge(df_output, df, on='Neighborhood', how='left')
df_temp.fillna(0, inplace=True)
df_output = drop_na_check(df_temp)

In [12]:
# housing
df = dfs['housing'].rename(columns={'Neighborhood_2010_HOOD': 'Neighborhood', 'Neighborhood_2010_INTPTLAT10': 'Latitude', 'Neighborhood_2010_INTPTLON10': 'Longitude'})
df = df.drop(columns=['Latitude', 'Longitude'])
# df = df[['Neighborhood_2010_HOOD', 'SNAP_All_csv_Median_Sale_Price_', 'SNAP_All_csv_2009_Median_Income', 'Est__Percent_Under_Poverty__201', 'SNAP_All_csv__Part_1__Major_Cri', 'SNAP_All_csv_Park_Space____of_l']]
# df.columns = ['Neighborhood', 'Median_Sale_Price', 'Median_Income', 'Under_Poverty %', 'Major Crime Count', 'Park Space %']

df_temp = pd.merge(df_output, df, on='Neighborhood', how='left')
df_temp.fillna(0, inplace=True)
df_output = drop_na_check(df_temp)

In [13]:
# parking rate
df = dfs['parking_meter_locations'].copy()
df['rate'] =  df['rate'].str.extract('(\d+)').astype(float)

append_to_df_output(df, '', 'parking_meter_count')
append_to_df_output(df, 'rate', 'parking_avg_rate')
df_output['parking_avg_rate'] = df_output['parking_avg_rate']/df_output['parking_meter_count']

In [14]:
# park and ride location
df = dfs['park_and_ride_locations']
append_to_df_output(df, '', 'park_and_ride_count')

In [15]:
# PGHSNAP
df = dfs['PGHSNAP']
# df = df[['Neighborhood', 'Population (2010)', 'Street Density (st. mi/area sq. mi)', 'Total Working Pop. (Age 16+) (2010)', 'Work at Home (2010)']]
# df.columns = ['Neighborhood', 'Population', 'Street Density', 'Total Working Pop', 'Work at Home %']

df_temp = pd.merge(df_output, df, on='Neighborhood', how='left')
df_output = df_temp

In [16]:
# save datasets
df_output.to_csv(folder_name+'/post_processed.csv', index=False)
df_output

,Neighborhood,Latitude,Longitude,arrests_count,bike_station_count,crash_count,firearm_seizure_count,Neighborhood_2010_SQMILES,SNAP_All_csv_Median_Home_Value_,Med__Val____00_in__10_Dollars_,...,Total Working Pop. (Age 16+) (2010),Commute to Work: Drive Alone (2010),Commute to Work: Carpool/Vanpool (2010),Commute to Work: Public Transportation (2010),Commute to Work: Taxi (2010),Commute to Work: Motorcycle (2010),Commute to Work: Bicycle (2010),Commute to Work: Walk (2010),Commute to Work: Other (2010),Work at Home (2010)
0,Allegheny Center,40.451577,-80.005340,1242.0,42.0,0.0,16.0,0.210,"$86,500.00","$109,535.00",...,386,0.277202,0.098446,0.168394,0.000000,0.000000,0.000000,0.437824,0.000000,0.018135
1,Allegheny West,40.450784,-80.014403,129.0,38.0,0.0,16.0,0.141,"$159,700.00","$202,228.00",...,151,0.231788,0.205298,0.158940,0.145695,0.000000,0.000000,0.258278,0.000000,0.000000
2,Allentown,40.419192,-79.992731,945.0,0.0,0.0,67.0,0.295,"$34,300.00","$43,434.00",...,824,0.469660,0.109223,0.379854,0.000000,0.008495,0.000000,0.000000,0.014563,0.018204
3,Arlington,40.413833,-79.963270,309.0,0.0,0.0,28.0,0.470,"$38,800.00","$49,132.00",...,673,0.739970,0.072808,0.135215,0.000000,0.000000,0.000000,0.019316,0.008915,0.023774
4,Arlington Heights,40.416880,-79.961521,154.0,23.0,0.0,8.0,0.132,"$45,000.00","$56,984.00",...,72,0.111111,0.000000,0.888889,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,Upper Lawrenceville,40.481811,-79.947774,189.0,19.0,0.0,11.0,0.405,"$35,900.00","$45,460.00",...,1163,0.567498,0.054170,0.250215,0.000000,0.000000,0.010318,0.052451,0.000000,0.065348
86,West End,40.440883,-80.035707,289.0,0.0,0.0,6.0,0.200,"$26,800.00","$33,937.00",...,64,0.484375,0.171875,0.156250,0.000000,0.187500,0.000000,0.000000,0.000000,0.000000
87,West Oakland,40.441244,-79.962611,324.0,41.0,0.0,13.0,0.215,"$47,500.00","$60,149.00",...,834,0.354916,0.009592,0.220624,0.000000,0.000000,0.000000,0.407674,0.007194,0.000000
88,Westwood,40.432010,-80.053037,226.0,0.0,0.0,10.0,0.692,"$72,200.00","$91,427.00",...,1563,0.630198,0.113884,0.128599,0.000000,0.011516,0.000000,0.087012,0.000000,0.028791
